In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from chroma_server.db.clickhouse import Clickhouse
from chroma_server.index.hnswlib import Hnswlib

from chroma_server.utils.sampling import *

In [5]:
db = Clickhouse()
model_space = "default_scope"

training_dataset_name = "waymo-phx-train-hc"
inference_dataset_name = "waymo-sf-val-hc"

training_data = db.fetch(
    where={"model_space": model_space, "dataset": training_dataset_name}
)
inference_data = db.fetch(
    where={"model_space": model_space, "dataset": inference_dataset_name}
)

ann_index = Hnswlib()

ann_index.run(
    model_space=model_space,
    uuids=training_data["uuid"].tolist() + inference_data["uuid"].tolist(),
    embeddings=training_data["embedding"].tolist() + inference_data["embedding"].tolist(),
    num_threads=8
)

time to fetch 1677402 embeddings:  7.514243125915527
time to fetch 205777 embeddings:  1.2197113037109375


# Score and Store

In [8]:
score_and_store(
    training_dataset_name=training_dataset_name,
    inference_dataset_name=inference_dataset_name,
    db_connection=db,
    ann_index=ann_index,
    model_space=model_space,
)

time to fetch 1677402 embeddings:  11.789789915084839
time to fetch 205777 embeddings:  0.9775769710540771


# Get Samples

In [12]:
sample_proportions = {
    "activation_uncertainty": 0.3,
    "boundary_uncertainty": 0.3,
    "representative_cluster_outlier": 0.2,
    "random": 0.2,
}

samples = get_sample(dataset_name=inference_dataset_name, n_samples=1000, sample_proportions=sample_proportions, db_connection=db, model_space=model_space)